In [ ]:
import requests
import pandas as pd
import xmltodict

# MySQL에 약국 데이터 저장
def save_to_mysql(data):
    import pymysql
    connection = pymysql.connect(host='데이터베이스주소', 
                                   port=3306,
                                   user='아이디', 
                                   passwd='비밀번호', 
                                   db='medical_facilities',
                                   charset='utf8')  # 한글 깨짐 방지
        
    cursor = connection.cursor(pymysql.cursors.DictCursor)

    # 새 데이터 삽입 쿼리 (칼럼 순서가 schema와 일치하도록 수정)
    insert_query = """
        INSERT INTO pharmacy_data (
            dutyAddr, dutyName, dutyImg, dutyInf, dutyMapimg, dutyTel1, 
            dutyTime1s, dutyTime1c, dutyTime2s, dutyTime2c,
            dutyTime3s, dutyTime3c, dutyTime4s, dutyTime4c, dutyTime5s, dutyTime5c, 
            dutyTime6s, dutyTime6c, dutyTime7s, dutyTime7c, dutyTime8s, dutyTime8c, 
            hpid, postCdn1, postCdn2, wgs84Lat, wgs84Lon, dutyEtc
        ) 
        VALUES (
            %s, %s, %s, %s, %s, %s, 
            %s, %s, %s, %s,
            %s, %s, %s, %s, %s, %s, 
            %s, %s, %s, %s, %s, %s, 
            %s, %s, %s, %s, %s, %s
        )
    """
    cursor.executemany(insert_query, data)
    connection.commit()
    cursor.close()
    connection.close()


# 시간 데이터를 VARCHAR(4) 형식으로 처리
def convert_to_time_format(time_str):
    if time_str and len(time_str) == 4:
        return time_str  # 그대로 HHMM 형식으로 반환 (예: 1800 -> '1800')
    return None  # 시간 값이 없으면 None 반환


# API 호출 및 데이터 수집
def fetch_and_store_data(servicekey, pageNo):
    from pprint import pprint  # dict 가독성 좋게 출력

    # API 요청 파라미터 구성
    params = {
        'ServiceKey': servicekey,
        'numOfRows': 1000,
        'pageNo': pageNo
    }
    response = requests.get(api_endpoint, params=params)

    # API 요청 결과 파싱
    data_to_insert = []
    if response.status_code == 200:
        root = xmltodict.parse(response.content)  # XML을 JSON 형식으로 변환
        items = root['response']['body']['items']['item']  # items 내의 각 항목을 추출
        for item in items:
            data_to_insert.append((
                item['dutyAddr'],  # dutyAddr (주소)
                item['dutyName'],  # dutyName (기관명)
                item.get('dutyImg', None),  # dutyImg (기관 이미지)
                item.get('dutyInf', None),  # dutyInf (기관 설명)
                item.get('dutyMapimg', None),  # dutyMapimg (간이 약도)
                item['dutyTel1'],  # dutyTel1 (전화번호)
                convert_to_time_format(item.get('dutyTime1s', None)),  # dutyTime1s (월요일 시작 시간)
                convert_to_time_format(item.get('dutyTime1c', None)),  # dutyTime1c (월요일 종료 시간)
                convert_to_time_format(item.get('dutyTime2s', None)),  # dutyTime2s (화요일 시작 시간)
                convert_to_time_format(item.get('dutyTime2c', None)),  # dutyTime2c (화요일 종료 시간)
                convert_to_time_format(item.get('dutyTime3s', None)),  # dutyTime3s (수요일 시작 시간)
                convert_to_time_format(item.get('dutyTime3c', None)),  # dutyTime3c (수요일 종료 시간)
                convert_to_time_format(item.get('dutyTime4s', None)),  # dutyTime4s (목요일 시작 시간)
                convert_to_time_format(item.get('dutyTime4c', None)),  # dutyTime4c (목요일 종료 시간)
                convert_to_time_format(item.get('dutyTime5s', None)),  # dutyTime5s (금요일 시작 시간)
                convert_to_time_format(item.get('dutyTime5c', None)),  # dutyTime5c (금요일 종료 시간)
                convert_to_time_format(item.get('dutyTime6s', None)),  # dutyTime6s (토요일 시작 시간)
                convert_to_time_format(item.get('dutyTime6c', None)),  # dutyTime6c (토요일 종료 시간)
                convert_to_time_format(item.get('dutyTime7s', None)),  # dutyTime7s (일요일 시작 시간)
                convert_to_time_format(item.get('dutyTime7c', None)),  # dutyTime7c (일요일 종료 시간)
                convert_to_time_format(item.get('dutyTime8s', None)),  # dutyTime8s (공휴일 시작 시간)
                convert_to_time_format(item.get('dutyTime8c', None)),  # dutyTime8c (공휴일 종료 시간)
                item['hpid'],  # hpid (기관 ID)
                item['postCdn1'],  # postCdn1 (우편번호1)
                item['postCdn2'],  # postCdn2 (우편번호2)
                item['wgs84Lat'],  # wgs84Lat (위도)
                item['wgs84Lon'],  # wgs84Lon (경도)
                item.get('dutyEtc', None)  # dutyEtc (비고, NULL일 수 있음)
            ))

    pprint(data_to_insert)  # 데이터를 출력하여 확인
    # MySQL에 데이터 저장
    save_to_mysql(data_to_insert)
    print('저장 완료')


In [ ]:
# API 호출 및 데이터 수집
def fetch_and_store_data(servicekey, pageNo):
    from pprint import pprint  # dict 가독성 좋게 출력

    # API 요청 파라미터 구성
    params = {
        'ServiceKey': servicekey,
        'numOfRows': 1000,
        'pageNo': pageNo
    }
    response = requests.get(api_endpoint, params=params)

    # API 요청 결과 파싱
    data_to_insert = []
    if response.status_code == 200:
        root = xmltodict.parse(response.content)  # XML을 JSON 형식으로 변환
        items = root['response']['body']['items']['item']
        for item in items:
            data_to_insert.append((
                item['addr'],  # dutyAddr (주소)
                item['yadmNm'],  # dutyName (기관명)
                item.get('dutyImg', None),  # dutyImg (기관 이미지)
                item.get('dutyMapimg', None),  # dutyMapimg (간이 약도)
                None,  # dutyInf (부서명, department_name이 제거됨)
                item.get('telno', None),  # dutyTel1 (전화번호)
                item.get('dutyTime1s', None),  # dutyTime1s (월요일 시작 시간)
                item.get('dutyTime1c', None),  # dutyTime1c (월요일 종료 시간)
                item.get('dutyTime2s', None),  # dutyTime2s (화요일 시작 시간)
                item.get('dutyTime2c', None),  # dutyTime2c (화요일 종료 시간)
                item.get('dutyTime3s', None),  # dutyTime3s (수요일 시작 시간)
                item.get('dutyTime3c', None),  # dutyTime3c (수요일 종료 시간)
                item.get('dutyTime4s', None),  # dutyTime4s (목요일 시작 시간)
                item.get('dutyTime4c', None),  # dutyTime4c (목요일 종료 시간)
                item.get('dutyTime5s', None),  # dutyTime5s (금요일 시작 시간)
                item.get('dutyTime5c', None),  # dutyTime5c (금요일 종료 시간)
                item.get('dutyTime6s', None),  # dutyTime6s (토요일 시작 시간)
                item.get('dutyTime6c', None),  # dutyTime6c (토요일 종료 시간)
                item.get('dutyTime7s', None),  # dutyTime7s (일요일 시작 시간)
                item.get('dutyTime7c', None),  # dutyTime7c (일요일 종료 시간)
                item.get('dutyTime8s', None),  # dutyTime8s (공휴일 시작 시간)
                item.get('dutyTime8c', None),  # dutyTime8c (공휴일 종료 시간)
                item.get('hpid', None),  # hpid (기관 ID)
                item.get('postCdn1', None),  # postCdn1 (우편번호1)
                item.get('postCdn2', None),  # postCdn2 (우편번호2)
                item.get('XPos', None),  # wgs84Lat (위도)
                item.get('YPos', None)   # wgs84Lon (경도)
            ))

    pprint(data_to_insert)  # 데이터를 출력하여 확인
    # MySQL에 데이터 저장
    save_to_mysql(data_to_insert)
    print('저장 완료')